# Neutron Event Analyzer Tutorial

This tutorial demonstrates how to use the `neutron_event_analyzer` package to process and analyze neutron event camera data. The package processes `.empirphot` and `.empirevent` files, associates photons with events, computes ellipticity, and generates diagnostic plots.

## Setup

Ensure you have installed the package:

```bash
pip install neutron_event_analyzer
```

You need a data folder with the following structure:

```
data_folder/
├── photonFiles/
│   └── *.empirphot
└── eventFiles/
    └── *.empirevent
```

Additionally, provide the path to the `export` directory containing the `empir_export_events` and `empir_export_photons` binaries.

## Prerequisites

Let's import the necessary module and set up the environment.

In [ ]:
from neutron_event_analyzer import Analyse
import numpy as np

# Define paths (replace with your actual paths)
data_folder = '/path/to/your/data_folder'
export_dir = '/path/to/your/export'

## Step 1: Initialize the Analyser

Create an `Analyse` object, specifying the data folder and export directory.

In [ ]:
analyser = Analyse(data_folder=data_folder, export_dir=export_dir, n_threads=4)
print("Analyser initialized.")

## Step 2: Load Event and Photon Data

Load the event and photon files into DataFrames. You can specify custom glob patterns if needed.

In [ ]:
# Load events
analyser.load_events()

# Load photons
analyser.load_photons()

## Step 3: Associate Photons to Events

Associate photons to events based on spatial and temporal proximity.

In [ ]:
analyser.associate(time_norm_ns=1.0, spatial_norm_px=1.0, dSpace_px=np.inf, verbosity=1)

## Step 4: Compute Ellipticity

Compute ellipticity and related metrics for multi-photon events.

In [ ]:
analyser.compute_ellipticity(verbosity=1)

## Step 5: Retrieve Combined DataFrame

Get the DataFrame containing associated photon and event data, including ellipticity metrics.

In [ ]:
combined_df = analyser.get_combined_dataframe()
combined_df.head()

## Step 6: Generate Diagnostic Plots

Create four diagnostic plots to visualize event data.

In [ ]:
analyser.plot_four(name='MyRun', min_n=1, max_n=1000, min_psd=1e-10, max_psd=1)

## Notes

- Ensure the `export_dir` contains the required binaries.
- Adjust `n_threads` based on your system's capabilities.
- The `plot_four` method generates hexbin plots and a histogram for event analysis.
- For large datasets, processing may take time; monitor progress via the `tqdm` bars.

For more details, refer to the [README](README.md) file in the package directory.